In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Data EDA & Preprocessing

In [ ]:
# package load
import matplotlib.pyplot as plt
import seaborn as sns

### 1.1 Train Data

In [ ]:
# data load
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
train

In [ ]:
# info ccheck
display(train.info())


# stat check
display(train.describe().T)

# type check
display(train.dtypes.value_counts())

# null column
display((train.isnull().sum()>=1).sort_values(ascending = False))

# # unique value check
# display(train.nunique())

- too many null value : Alley FireplaceQu, PoolQC, Fence, MiscFeature

In [ ]:
# "Id" drop
train_pp = train.drop('Id',axis = 1)
train_pp

In [ ]:
# too many null column
drop_col = (train_pp.isnull().sum().sort_values(ascending = False)).head(5).index
drop_col

In [ ]:
# too many null column drop
train_pp = train_pp.drop(drop_col,axis = 1)
train_pp

In [ ]:
# dist check
sns.distplot(train_pp["SalePrice"],
             hist = True, kde = True, color = 'green')
plt.show()

- need scaleing

In [ ]:
# log transform
train_pp["SalePrice"] = np.log1p(train_pp["SalePrice"])

# dist check
sns.distplot(train_pp["SalePrice"],
             hist = True, kde = True, color = 'green')
plt.show()

In [ ]:
# int, float variable fillna
train_pp.fillna(train_pp.mean(), inplace = True)

# info check
display(train_pp.info())

### 1.2 Test Data Check

In [ ]:
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test

In [ ]:
# info ccheck
display(test.info())


# stat check
display(test.describe().T)

# type check
display(test.dtypes.value_counts())

# null column
display((test.isnull().sum()>=1).sort_values(ascending = False))

# # unique value check
# display(train.nunique())

In [ ]:
# "Id" drop
test_pp = test.drop('Id',axis = 1)
test_pp

In [ ]:
# too many null column
drop_col = (test_pp.isnull().sum().sort_values(ascending = False)).head(5).index
drop_col

In [ ]:
# too many null column drop
test_pp = test_pp.drop(drop_col,axis = 1)
test_pp

In [ ]:
# int, float variable fillna
test_pp.fillna(test_pp.mean(), inplace = True)

# info check
display(test_pp.info())

### 1.3 Train vs Test

In [ ]:
def check_train_test_column_values(train, test, column):
    # 함수 정의: 두 데이터 테이블과 특정 컬럼을 집어 넣으면 value를 비교하여 출력하는 함수
    print("{} Column에 대한 train_test_values_check 입니다=======================".format(column))
    # Train/Test set의 입력 칼럼의 value를 set으로 받아줍니다.
    train_colset = set(train[column])
    test_colset  = set(test[column])

    # Train/Test-set 고유한 value의 개수를 구함.
    print(f"Train-set에 있는 고유한 value 개수 : {len(train_colset)}")
    print(f"Test-set에 있는 고유한 value 개수 : {len(test_colset)}")

    # Train/Test-set 모두에 포함되어있는 value를 구함.
    print('='* 80)
    common_colset = train_colset.intersection(test_colset)
    print(f"Train/Test-set에 공통으로 포함되어 있는 value 개수 : {len(common_colset)}")
    if len(common_colset) > 100: # value가 너무 많으면 어차피 보기 힘드므로 출력을 pass 합니다
        pass
    else:
        try: # int나 float은 sorted가 먹지 않기 때문에 try except로 넣어줍니다.
            print(f"Train/Test-set에 공통으로 포함되어 있는 value : {sorted(common_colset)}")
        except:
            print(f"Train/Test-set에 공통으로 포함되어 있는 value : {(common_colset)}")

    # Train-set에만 있는 value를 구함.
    print('='* 80)
    train_only_colset = train_colset.difference(test_colset)
    print(f"Train-set에만 있는 value는 총 {len(train_only_colset)} 개 입니다.")
    if len(train_only_colset) > 100:
        pass
    else:
        try:
            print(f"Train-set에만 있는 value는 : {sorted(train_only_colset)}")
        except:
            print(f"Train-set에만 있는 value는 : {(train_only_colset)}")

    # Test-set에만 있는 value를 구함.
    print('='* 80)
    test_only_colset = test_colset.difference(train_colset)
    print(f"Test-set에만 있는 value는 총 {len(test_only_colset)} 개 입니다.")
    if len(test_only_colset) > 100:
        pass
    else:
        try:
            print(f"Test-set에만 있는 value는 : {sorted(test_only_colset)}")
        except:
            print(f"Test-set에만 있는 value는 : {(test_only_colset)}")
    print(" ")

In [ ]:
# obj col find
obj_cols = []
for col in train_pp.columns:
    if train_pp[col].dtypes == 'object':
        obj_cols.append(col)

In [ ]:
for col in obj_cols:
    check_train_test_column_values(train_pp, test_pp, col)

- possible get_dummies

### 1.4 one-hot Encoding

In [ ]:
# final dataset_ int&float variable
train_fin = train_pp.drop(obj_cols,axis = 1)
test_fin = test_pp.drop(obj_cols,axis = 1)


display(train_fin.info())
display(test_fin.info())

In [ ]:
# concat data to one-hot encoding

concat_data = pd.concat([train_pp[obj_cols],test_pp[obj_cols]],axis = 0)
concat_data[:1460] ## train : 1460, test : 1459

In [ ]:
# get_dummy

dummy_data = pd.get_dummies(concat_data)
dummy_data.shape

In [ ]:
# train, test split

train_dummy = dummy_data[:1460]
test_dummy = dummy_data[1460:]

display(train_dummy.shape)
display(test_dummy.shape)

In [ ]:
# data_merge
train_fin = pd.concat([train_fin,train_dummy],axis= 1)
test_fin = pd.concat([test_fin,test_dummy],axis= 1)

display(train_fin.shape)
display(test_fin.shape)

## 2. Model - Linear, Lasso, ridge

In [ ]:
# data 

X = train_fin.drop("SalePrice", axis = 1)
y = train_fin.SalePrice
test = test_fin
display(X)
display(y)
display(test)

In [ ]:
# data split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state = 42)

In [ ]:
# metric 
from sklearn.metrics import mean_squared_error

def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test , pred)
    rmse = np.sqrt(mse)
    print('{0} log transform RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

In [ ]:
# model load

from sklearn.linear_model import LinearRegression, Ridge, Lasso

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

In [ ]:
def get_top_bottom_coef(model):
    # coef_ 속성을 기반으로 Series 객체를 생성. index는 컬럼명. 
    coef = pd.Series(model.coef_, index=X_train.columns)
    
    # + 상위 10개 , - 하위 10개 coefficient 추출하여 반환.
    coef_high = coef.sort_values(ascending=False).head(10)
    coef_low = coef.sort_values(ascending=False).tail(10)
    return coef_high, coef_low

In [ ]:
def visualize_coefficient(models):
    # 3개 회귀 모델의 시각화를 위해 3개의 컬럼을 가지는 subplot 생성
    fig, axs = plt.subplots(figsize=(24,10),nrows=1, ncols=3)
    fig.tight_layout() 
    # 입력인자로 받은 list객체인 models에서 차례로 model을 추출하여 회귀 계수 시각화. 
    for i_num, model in enumerate(models):
        # 상위 10개, 하위 10개 회귀 계수를 구하고, 이를 판다스 concat으로 결합. 
        coef_high, coef_low = get_top_bottom_coef(model)
        coef_concat = pd.concat( [coef_high , coef_low] )
        # 순차적으로 ax subplot에 barchar로 표현. 한 화면에 표현하기 위해 tick label 위치와 font 크기 조정. 
        axs[i_num].set_title(model.__class__.__name__+' Coeffiecents', size=25)
        axs[i_num].tick_params(axis="y",direction="in", pad=-120)
        for label in (axs[i_num].get_xticklabels() + axs[i_num].get_yticklabels()):
            label.set_fontsize(22)
        sns.barplot(x=coef_concat.values, y=coef_concat.index , ax=axs[i_num])

# 앞 예제에서 학습한 lr_reg, ridge_reg, lasso_reg 모델의 회귀 계수 시각화.    
models = [lr_reg, ridge_reg, lasso_reg]
visualize_coefficient(models)

In [ ]:
# cross_val_score, cv fit

from sklearn.model_selection import cross_val_score

def get_avg_rmse_cv(models):
    for model in models:
        # 분할하지 않고 전체 데이터로 cross_val_score( ) 수행. 모델별 CV RMSE값과 평균 RMSE 출력
        rmse_list = np.sqrt(-cross_val_score(model, X, y,
                                             scoring="neg_mean_squared_error", cv = 5))
        rmse_avg = np.mean(rmse_list)
        print('\n{0} CV RMSE 값 리스트: {1}'.format( model.__class__.__name__, np.round(rmse_list, 3)))
        print('{0} CV 평균 RMSE 값: {1}'.format( model.__class__.__name__, np.round(rmse_avg, 3)))

# 앞 예제에서 학습한 lr_reg, ridge_reg, lasso_reg 모델의 CV RMSE값 출력           
models = [lr_reg, ridge_reg, lasso_reg]
get_avg_rmse_cv(models)

In [ ]:
# parameter tune

from sklearn.model_selection import GridSearchCV

def get_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid=params, 
                              scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X, y)
    rmse = np.sqrt(-1* grid_model.best_score_)
    print('{0} 5 CV 시 최적 평균 RMSE 값: {1}, 최적 alpha:{2}'.format(model.__class__.__name__,
                                        np.round(rmse, 4), grid_model.best_params_))
    return grid_model.best_estimator_

ridge_params = { 'alpha':[0.05, 0.1, 1, 5, 8, 10, 12, 15, 20] }
lasso_params = { 'alpha':[0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1,5, 10] }
best_rige = get_best_params(ridge_reg, ridge_params)
best_lasso = get_best_params(lasso_reg, lasso_params)

In [ ]:
# valid

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge(alpha=12)
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

# 모든 모델의 RMSE 출력
models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

# 모든 모델의 회귀 계수 시각화 
models = [lr_reg, ridge_reg, lasso_reg]
visualize_coefficient(models)

## 3. model upgrade

### 3.1 data dist check

In [ ]:
from scipy.stats import skew

# non-object variable 
features_index = train_pp.dtypes[train_pp.dtypes != 'object'].index

# skew check
skew_features = train_pp[features_index].apply(lambda x : skew(x))

# skew > 1
skew_features_top_train = skew_features[skew_features > 1]
print(skew_features_top_train.sort_values(ascending=False))

**독립변수들의 왜곡이 심해도 회귀 성능에 영향을 줄 수 있기 때문에 train, test셋의 공통된 왜곡이 심한 변수들을 로그 변환을 취해준다.**

In [ ]:
# non-object variable 
features_index = test_pp.dtypes[test_pp.dtypes != 'object'].index

# skew check
skew_features = test_pp[features_index].apply(lambda x : skew(x))

# skew > 5
skew_features_top_test = skew_features[skew_features > 1]
print(skew_features_top_test.sort_values(ascending=False))

In [ ]:
common_colset = skew_features_top_train.index.intersection(skew_features_top_test.index)
common_colset

In [ ]:
train_pp[common_colset] = np.log1p(train_pp[common_colset])
test_pp[common_colset] = np.log1p(test_pp[common_colset])

In [ ]:
# final dataset_ int&float variable
train_fin = train_pp.drop(obj_cols,axis = 1)
test_fin = test_pp.drop(obj_cols,axis = 1)


display(train_fin.info())
display(test_fin.info())

In [ ]:
# concat data to one-hot encoding

concat_data = pd.concat([train_pp[obj_cols],test_pp[obj_cols]],axis = 0)
concat_data[:1460] ## train : 1460, test : 1459

In [ ]:
# get_dummy

dummy_data = pd.get_dummies(concat_data)
dummy_data.shape

In [ ]:
# train, test split

train_dummy = dummy_data[:1460]
test_dummy = dummy_data[1460:]

display(train_dummy.shape)
display(test_dummy.shape)

In [ ]:
# data_merge
train_fin = pd.concat([train_fin,train_dummy],axis= 1)
test_fin = pd.concat([test_fin,test_dummy],axis= 1)

display(train_fin.shape)
display(test_fin.shape)

In [ ]:
# data 

X = train_fin.drop("SalePrice", axis = 1)
y = train_fin.SalePrice
test = test_fin
display(X)
display(y)
display(test)

In [ ]:
# data split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state = 42)

In [ ]:
# 피처들을 로그 변환 후 다시 최적 하이퍼 파라미터와 RMSE 출력
ridge_params = { 'alpha':[0.05, 0.1, 1, 5, 8, 10, 12, 15, 20] }
lasso_params = { 'alpha':[0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1,5, 10] }
best_ridge = get_best_params(ridge_reg, ridge_params)
best_lasso = get_best_params(lasso_reg, lasso_params)

In [ ]:
# 앞의 최적화 alpha값으로 학습데이터로 학습, 테스트 데이터로 예측 및 평가 수행. 
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge(alpha=5)
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

# 모든 모델의 RMSE 출력
models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

# 모든 모델의 회귀 계수 시각화 
models = [lr_reg, ridge_reg, lasso_reg]
visualize_coefficient(models)

### 3.2 outlier check

In [ ]:
plt.scatter(x = train_pp['GrLivArea'], y = train_pp['SalePrice'])
plt.ylabel('SalePrice', fontsize=15)
plt.xlabel('GrLivArea', fontsize=15)
plt.show()

In [ ]:
# GrLivArea와 SalePrice 모두 로그 변환되었으므로 이를 반영한 조건 생성. 
cond1 = train_pp['GrLivArea'] > np.log1p(4000)
cond2 = train_pp['SalePrice'] < np.log1p(500000)
outlier_index = train_pp[cond1 & cond2].index

print('아웃라이어 레코드 index :', outlier_index.values)
print('아웃라이어 삭제 전 house_df_ohe shape:', train_pp.shape)
# DataFrame의 index를 이용하여 아웃라이어 레코드 삭제. 
train_pp.drop(outlier_index , axis=0, inplace=True)
print('아웃라이어 삭제 후 house_df_ohe shape:', train_pp.shape)

In [ ]:
plt.scatter(x = train_pp['1stFlrSF'], y = train_pp['SalePrice'])
plt.ylabel('SalePrice', fontsize=15)
plt.xlabel('1stFlrSF', fontsize=15)
plt.show()

In [ ]:
# final dataset_ int&float variable
train_fin = train_pp.drop(obj_cols,axis = 1)
test_fin = test_pp.drop(obj_cols,axis = 1)


display(train_fin.info())
display(test_fin.info())

In [ ]:
# concat data to one-hot encoding

concat_data = pd.concat([train_pp[obj_cols],test_pp[obj_cols]],axis = 0)
concat_data[:1458] ## train : 1458, test : 1459

In [ ]:
# get_dummy

dummy_data = pd.get_dummies(concat_data)
dummy_data.shape

In [ ]:
# train, test split

train_dummy = dummy_data[:1458]
test_dummy = dummy_data[1458:]

display(train_dummy.shape)
display(test_dummy.shape)

In [ ]:
# data_merge
train_fin = pd.concat([train_fin,train_dummy],axis= 1)
test_fin = pd.concat([test_fin,test_dummy],axis= 1)

display(train_fin.shape)
display(test_fin.shape)

In [ ]:
# data 

X = train_fin.drop("SalePrice", axis = 1)
y = train_fin.SalePrice
test = test_fin
display(X)
display(y)
display(test)

In [ ]:
# data split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state = 42)

In [ ]:
# 피처들을 로그 변환 후 다시 최적 하이퍼 파라미터와 RMSE 출력
ridge_params = { 'alpha':[0.05, 0.1, 1, 5, 8, 10, 12, 15, 20] }
lasso_params = { 'alpha':[0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1,5, 10] }
best_ridge = get_best_params(ridge_reg, ridge_params)
best_lasso = get_best_params(lasso_reg, lasso_params)

In [ ]:
# 앞의 최적화 alpha값으로 학습데이터로 학습, 테스트 데이터로 예측 및 평가 수행. 
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge(alpha=8)
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

# 모든 모델의 RMSE 출력
models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

# 모든 모델의 회귀 계수 시각화 
models = [lr_reg, ridge_reg, lasso_reg]
visualize_coefficient(models)

In [ ]:
# sub
sub = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')


final_model = lasso_reg

pred = final_model.predict(test)

sub.loc[:,"SalePrice"] = np.expm1(pred)

display(sub)

sub.to_csv("submission_lasso_3_2.csv",index = False)

## 4. model - tree regressor - XGB, LGBM

In [ ]:
from xgboost import XGBRegressor

xgb_params = {'n_estimators':[1000]}
xgb_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05, 
                       colsample_bytree=0.5, subsample=0.8)
best_xgb = get_best_params(xgb_reg, xgb_params)

In [ ]:
from lightgbm import LGBMRegressor

lgbm_params = {'n_estimators':[1000]}
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=4, 
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)
best_lgbm = get_best_params(lgbm_reg, lgbm_params)

In [ ]:
# sub
sub = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

lgbm_reg.fit(X_train, y_train)
final_model = lgbm_reg

pred = final_model.predict(test)

sub.loc[:,"SalePrice"] = np.expm1(pred)

display(sub)

sub.to_csv("submission_lgbm_reg.csv",index = False)

## 5. blend model

In [ ]:
## ridge & lasso

def get_rmse_pred(preds):
    for key in preds.keys():
        pred_value = preds[key]
        mse = mean_squared_error(y_test , pred_value)
        rmse = np.sqrt(mse)
        print('{0} 모델의 RMSE: {1}'.format(key, rmse))

# 개별 모델의 학습
ridge_reg = Ridge(alpha=8)
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

# 개별 모델 예측
ridge_pred = ridge_reg.predict(X_test)
lasso_pred = lasso_reg.predict(X_test)

# 개별 모델 예측값 혼합으로 최종 예측값 도출
pred = 0.5 * ridge_pred + 0.5* lasso_pred
preds = {'최종 혼합': pred,
         'Ridge': ridge_pred,
         'Lasso': lasso_pred}
#최종 혼합 모델, 개별모델의 RMSE 값 출력
get_rmse_pred(preds)

In [ ]:
# sub
sub = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

ridge_pred = ridge_reg.predict(test)
lasso_pred = lasso_reg.predict(test)

pred = 0.5 * ridge_pred + 0.5* lasso_pred

sub.loc[:,"SalePrice"] = np.expm1(pred)

display(sub)

sub.to_csv("submission_blend_ridge_lasso.csv",index = False)

In [ ]:
## XGB & LGBM Blend

xgb_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05, 
                       colsample_bytree=0.5, subsample=0.8)
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=4, 
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)
xgb_reg.fit(X_train, y_train)
lgbm_reg.fit(X_train, y_train)

xgb_pred = xgb_reg.predict(X_test)
lgbm_pred = lgbm_reg.predict(X_test)

pred = 0.5 * xgb_pred + 0.5 * lgbm_pred
preds = {'최종 혼합': pred,
         'XGBM': xgb_pred,
         'LGBM': lgbm_pred}
        
get_rmse_pred(preds)

In [ ]:
# sub
sub = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

xgb_pred = ridge_reg.predict(test)
lgbm_pred = lgbm_reg.predict(test)

pred = 0.5 * xgb_pred + 0.5* lgbm_pred

sub.loc[:,"SalePrice"] = np.expm1(pred)

display(sub)

sub.to_csv("submission_blend_XGB_LGBM.csv",index = False)

## 6. Stacking ensemble

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [ ]:
# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환. 
X_train_n = X_train.values
X_test_n = X_test.values
y_train_n = y_train.values

# 각 개별 기반(Base)모델이 생성한 학습용/테스트용 데이터 반환. 
ridge_train, ridge_test = get_stacking_base_datasets(ridge_reg, X_train_n, y_train_n, X_test_n, 5)
lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg, X_train_n, y_train_n, X_test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_reg, X_train_n, y_train_n, X_test_n, 5)  
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_reg, X_train_n, y_train_n, X_test_n, 5)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.  
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, 
                                      xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, 
                                     xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 라쏘 모델을 적용. 
meta_model_lasso = Lasso(alpha=0.0005)

#기반 모델의 예측값을 기반으로 새롭게 만들어진 학습 및 테스트용 데이터로 예측하고 RMSE 측정.
meta_model_lasso.fit(Stack_final_X_train, y_train)
final = meta_model_lasso.predict(Stack_final_X_test)
mse = mean_squared_error(y_test , final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RMSE 값은:', rmse)

In [ ]:
# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환. 
X_train_n = X_train.values
y_train_n = y_train.values
test_n = test.values


# 각 개별 기반(Base)모델이 생성한 학습용/테스트용 데이터 반환. 
ridge_train, ridge_test = get_stacking_base_datasets(ridge_reg, X_train_n, y_train_n, test_n, 5)
lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg, X_train_n, y_train_n, test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_reg, X_train_n, y_train_n, test_n, 5)  
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_reg, X_train_n, y_train_n, test_n, 5)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.  
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, 
                                      xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, 
                                     xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 라쏘 모델을 적용. 
meta_model_lasso = Lasso(alpha=0.0005)

#기반 모델의 예측값을 기반으로 새롭게 만들어진 학습 및 테스트용 데이터로 예측하고 RMSE 측정.
meta_model_lasso.fit(Stack_final_X_train, y_train)


# sub
sub = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

pred = meta_model_lasso.predict(Stack_final_X_test)

sub.loc[:,"SalePrice"] = np.expm1(pred)

display(sub)

sub.to_csv("submission_blend_stacking.csv",index = False)